In [1]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

from astropy.table import Table, Column, vstack

import numpy as np 
import matplotlib.pyplot as plt
    
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

from astroML.stats import binned_statistic_2d

from cap_loess_2d import loess_2d
from cap_plot_velfield import plot_velfield

from mpl_toolkits.axes_grid1 import make_axes_locatable

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)

from hsc_massive import \
    s16a_path, \
    sample_selection, \
    prepare_sed, \
    catalog_summary, \
    mass_function, \
    smhm, \
    plotting

# Catalogs for Enia

For quick computation using the same Universe Machine Multidark simulations as we are using for your project. 

She will be using halo tools to compute 
1. the correlation function between halos with Mhalo>10^14 and DESI like galaxies
2. the correlation function between halos with Mhalo>10^14 and dark matter particles.

Catalogs:
1. take the galaxies in the UM mock and make a stellar mass cut such that the cumulative number density is about 6e-3 (with h=1). Send Enia the galaxy catalog (she just needs x,y,z positions, central satellite flag).
2. x, y, z, Mhalo for halos with Mhalo>1e14
3. x,y,z for dark matter particles

In [38]:
um_smdpl_box = 400.0 # Mpc 
um_smdpl_h0 = 1.0 
um_smdpl_vol = np.power(um_smdpl_box / um_smdpl_h0, 3.0)

print("# Volume of the UniverseMachine model: %d Mpc" % um_smdpl_vol)

# Volume of the UniverseMachine model: 64000000 Mpc


In [39]:
um_smdpl_vol * 6.0E-3

384000.0

In [40]:
# um_smdpl_dir = '/Users/song/data/massive/dr16a/um2/um2_new/um_smdpl_small'
um_smdpl_dir = '/Users/song/astro5/massive/dr16a/um2/um2_new/um_smdpl_small'
um_smdpl_short = 'sfr_catalog_insitu_exsitu_0.712400_final_logmp_11.npz'

um_smdpl_new_cen = np.load(
    os.path.join(um_smdpl_dir, um_smdpl_short))['centrals']
um_smdpl_new_sat = np.load(
    os.path.join(um_smdpl_dir, um_smdpl_short))['satellites']

print(len(um_smdpl_new_cen), len(um_smdpl_new_sat))

(3194800, 14478087)


In [42]:
um_smdpl_tab_cen = Table(um_smdpl_new_cen)
um_smdpl_tab_sat = Table(um_smdpl_new_sat)

um_smdpl_tab = vstack([um_smdpl_tab_cen, um_smdpl_tab_sat])
print(um_smdpl_tab.colnames)

['id', 'upid', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'm', 'mp', 'sm', 'icl', 'sfr']


In [44]:
um_smdpl_tab.write(
    os.path.join(um_smdpl_dir, 'sfr_catalog_insitu_exsitu_0.712400_final_logmp_11.fits'),
    format='fits', overwrite=True)

In [45]:
um_smdpl_tab.add_column(Column(data=(um_smdpl_tab['sm'] + um_smdpl_tab['icl']), 
                               name='ms_tot'))

## Catalog 1

In [47]:
um_smdpl_tab.sort('ms_tot')
um_smdpl_tab.reverse()

In [49]:
np.log10(um_smdpl_tab[384000]['ms_tot'])

10.623796983177254

In [51]:
um_smdpl_gal1 = um_smdpl_tab[:384000]

um_smdpl_gal1.write(os.path.join('/Users/song/Downloads/',
                                 'um_smdpl_logms_gal_top_4m.fits'))

## Catalog 2

In [52]:
halo_mask = ((um_smdpl_tab['upid'] == -1) & 
             (np.log10(um_smdpl_tab['mp']) >= 14.0))
print(sum(halo_mask))

1805


In [53]:
um_smdpl_gal2 = um_smdpl_tab[halo_mask]

um_smdpl_gal2.write(os.path.join('/Users/song/Downloads/',
                                 'um_smdpl_logmh_peak_14.0.fits'))

## Catalog 3: Particles

In [56]:
um_smdpl_10m = np.load('/Users/song/astro5/massive/dr16a/um2/um2_new/um_smdpl_particles_0.7124_10m_v.npy')
um_smdpl_50m = np.load('/Users/song/astro5/massive/dr16a/um2/um2_new/um_smdpl_particles_0.7124_50m_v.npy')

In [57]:
um_smdpl_10m_tab = Table(um_smdpl_10m)
um_smdpl_50m_tab = Table(um_smdpl_50m)

In [58]:
um_smdpl_10m_tab.write('/Users/song/Downloads/um_smdpl_particles_0.7124_10m_v.fits', 
                       format='fits', overwrite=True)
um_smdpl_50m_tab.write('/Users/song/Downloads/um_smdpl_particles_0.7124_50m_v.fits', 
                       format='fits', overwrite=True)